In [ ]:
%load_ext blackcellmagic

# Declarator for counting words

In [ ]:
def word_count(func, *args, **kwargs):
    def wrapper():
        counter = {}
        words = func(*args, **kwargs)
        for word in words:
            if word in counter:
                counter[word] += 1
            else:
                counter[word] = 1
        return func(*args, **kwargs), counter

    return wrapper

# Cosine distance to target

1. cosine(output, target)
2. vectorize
3. apply on batch
4. average

## Use tf.keras.losses.cosine_similarity for speed

-1 : identical
0 : 90 degree
1: 180 degree

y_true = [[0., 1.], [1., 1.], [1., 1.]]
y_pred = [[1., 0.], [1., 1.], [-1., -1.]]
loss = tf.keras.losses.cosine_similarity(y_true, y_pred, axis=1)
loss.numpy()

array([-0.       , -0.9999998,  0.9999998], dtype=float32)

In [ ]:
import tensorflow as tf

In [ ]:
y_true = [[0.0, 1.0], [1.0, 1.0], [1.0, 1.0], [0, 0]]
y_pred = [[1.0, 0.0], [1.0, 1.0], [-1.0, -1.0], [0, 0]]
loss = tf.keras.losses.cosine_similarity(y_true, y_pred, axis=1)
loss.numpy()

In [ ]:
import troubleshooting
from importlib import reload
reload(troubleshooting)
run = "lesion_batchsize_OS20/lesion_batchsize_OS20_r0004"
root = 'home/jupyter/triangle_model'
test = troubleshooting.Diagnosis(code_name = run, tf_root_override = root)


In [ ]:
test.eval("train_r100", "triangle", epoch=1000)